In [1]:
from ngsolve import *
from ngsolve.webgui import Draw

from netgen.occ import *

In [9]:
outer = Rectangle(2,2).Face()
outer.edges.name="outer"
outer.edges.Max(X).name = "r"
outer.edges.Min(X).name = "l"
outer.edges.Min(Y).name = "b"
outer.edges.Max(Y).name = "t"


inner = MoveTo(1, 0.9).Rectangle(0.3, 0.5).Face()
inner.edges.name="interface"
outer = outer - inner

inner.faces.name="inner"
inner.faces.col = (1, 0, 0)
outer.faces.name="outer"

geo = Glue([inner, outer])

mesh = Mesh(OCCGeometry(geo, dim=2).GenerateMesh(maxh=0.2))
Draw(mesh);


# Draw(geo)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [18]:
fes1 = H1(mesh, definedon="inner")
fes2 = H1(mesh, definedon="outer")
u1 = GridFunction(fes1, "u1")
u2 = GridFunction(fes2, "u2")
u1.Set(x*y)
u2.Set(-x*y)
total = u1 + u2
# Draw(u1, mesh)
# Draw(u2, mesh)
# Draw(total, mesh)

In [15]:
print(fes1.FreeDofs())

0: 11110000111110000000000000000000000000000000000001
50: 10000000000000000000000000000000000000000000000000
100: 00000000000000000000000000000000


In [16]:
print(fes2.FreeDofs())

0: 11111111111111111111111111111111111111111111111110
50: 01111111111111111111111111111111111111111111111111
100: 11111111111111111111111111111111


In [19]:
fes = H1(mesh, order=3, dirichlet="b|l|r")
u, v = fes.TnT()
gfu = GridFunction(fes)

In [22]:
term1 = u1 * v * dx(definedon=mesh.Materials("inner"))
term2 = 0.1 * v * ds(definedon=mesh.Boundaries("t"))
f = LinearForm(term1 + term2).Assemble()

In [28]:
f = LinearForm(u1*v*dx('inner') + 0.1*v*ds('t')).Assemble()

In [29]:
a = BilinearForm(fes)
a += grad(u)*grad(v)*dx
a.Assemble()
gfu.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec
Draw(gfu);

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [30]:
mesh.GetBoundaries()

('interface', 'interface', 'interface', 'interface', 'b', 'l', 'r', 't')

In [31]:
mesh.GetMaterials()

('inner', 'outer')

In [34]:


print(mesh.Materials("inner").Mask())
print(mesh.Materials("outer").Mask())
print(mesh.Materials("outer|inner").Mask())
print(mesh.Materials("[a-z]*").Mask())  # can use regexp
print(mesh.Boundaries('t|l').Mask())



0: 10
0: 01
0: 11
0: 11
0: 00000101


In [40]:
print(mesh.Materials("outer").Mask() )
print(~mesh.Materials("inner").Mask())

0: 01
0: 01


In [41]:
domain_values = {'inner': 3.7,  'outer': 1}
cf = mesh.MaterialCF(domain_values)
Draw(cf, mesh);

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [42]:
bdry_values = {'b': x, 'r': 2-y}
cf = mesh.BoundaryCF(bdry_values, default=0)

In [43]:
g = GridFunction(H1(mesh, order=2), name="bdry")
g.Set(cf, definedon=mesh.Boundaries("b|r"))
Draw(g)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene